In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import cv2
import os
from facenet_pytorch import MTCNN # ?? 
from emotiefflib.facial_analysis import EmotiEffLibRecognizer, get_model_list, EmotiEffLibRecognizerTorch

In [3]:
images_folder = '../data/CityInfant/BlackWhite'

In [7]:
df = pd.read_csv('../data/AU/pyafar_infants.csv')
df = df.iloc[:,[0,1,-2]]
df['imageCat'] = df['imageCat'].map({1: 0, 2: 1, 3: 2})
df.to_csv("../data/CityInfant/Validationinfo/labels.csv", index=False)

In [ ]:
image_names = df['Image'].tolist()
affect_labels = df['imageCat'].tolist()


151
151


In [12]:
# Split data 

X_train, X_temp, y_train, y_temp = train_test_split(image_names, affect_labels, test_size=0.3, stratify=affect_labels, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(len(X_train))
print(len(X_dev))
print(len(X_test))

105
23
23


In [54]:
# --- 2. Custom Dataset Class ---

class AffectDataset(Dataset):
    def __init__(self, image_names, labels, images_folder, transform=None):
        self.image_names = image_names
        self.labels = labels
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        image_path = os.path.join(self.images_folder, img_name)

        try:
            image = cv2.imread(image_path) # Load image with OpenCV
            if image is None:
                raise ValueError(f"Could not read image at: {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert color (if needed)
            image = Image.fromarray(image) #Convert to PIL Image
        except Exception as e:
            print(f"Error loading image: {image_path} - {e}")
            raise  # Re-raise the exception

        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [16]:
# Define data transformations
image_size = 224 # Or 260, depending on the EmotiEffLib model
data_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])

In [55]:
def collate_fn(batch):
    """
    Custom collate function to handle None values in the batch.
    """
    # Filter out None values
    batch = [data for data in batch if data is not None]

    if not batch:
        return None, None  # Return None for both images and labels if batch is empty

    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    # Check if images and labels are empty after filtering
    if not images or not labels:
        return None, None

    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)  # Ensure labels are torch.long
    return images, labels


In [31]:
# Create datasets
train_dataset = AffectDataset(X_train, y_train, images_folder, transform=data_transforms)
dev_dataset = AffectDataset(X_dev, y_dev, images_folder, transform=data_transforms)
test_dataset = AffectDataset(X_test, y_test, images_folder, transform=data_transforms)

# Create DataLoaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last = True)  # Adjust num_workers as needed
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last = True)
test_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last = True)

#### Load pre-trained model #1

In [18]:
model_name = get_model_list()[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# Load EmotiEffLib recognizer
recognizer = EmotiEffLibRecognizer(model_name=model_name, engine="torch", device=device)

# Access the underlying PyTorch model 
model = recognizer.model

In [26]:
# Fine tune -- modify classifier to our classes 

num_classes = len(np.unique(affect_labels))

in_features = recognizer.classifier_weights.shape[1]

# Replace the classifier with a new linear layer
model.classifier = nn.Linear(in_features, num_classes).to(device)

In [28]:
criterion = nn.CrossEntropyLoss() #Standard cross entropy loss
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
num_epochs = 10

In [32]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() 

        outputs = model(images) #Forward pass
        loss = criterion(outputs, labels) #Calculate the loss
        loss.backward() #Backpropagation
        optimizer.step() #Update weights

        running_loss += loss.item()

        #Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_acc = correct_predictions / total_samples

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

Epoch 1/10, Loss: 1.0109, Accuracy: 0.4792
Epoch 2/10, Loss: 0.8590, Accuracy: 0.6771
Epoch 3/10, Loss: 0.7894, Accuracy: 0.8229
Epoch 4/10, Loss: 0.7346, Accuracy: 0.8542
Epoch 5/10, Loss: 0.6445, Accuracy: 0.8438
Epoch 6/10, Loss: 0.6072, Accuracy: 0.8542
Epoch 7/10, Loss: 0.5972, Accuracy: 0.8333
Epoch 8/10, Loss: 0.5085, Accuracy: 0.8854
Epoch 9/10, Loss: 0.4913, Accuracy: 0.8646
Epoch 10/10, Loss: 0.4156, Accuracy: 0.8958


In [42]:
test_dataloader

In [58]:
# --- 10. Validation ---
model.eval() #Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad(): 
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images) #Forward pass

        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

print('Accuracy: ', correct/total)


ZeroDivisionError: division by zero